Deben crear 6 funciones para los endpoints que se consumirán en la API, recuerden que deben tener un decorador por cada una (@app.get(‘/’)).

In [ ]:
#importamos las librerias 
import pandas as pd 
import numpy as np
import ast
from collections import defaultdict
import calendar
import asyncio
import json 


In [ ]:
#cargamos la data
data = pd.read_csv("definido_movies_dataset.csv")

In [ ]:
data.head()

In [ ]:
#cambiamos el tipo de pada de pais 
data["production_companies"] = data["production_companies"].astype(str)
data["production_companies"].fillna("Sin datos")

In [ ]:
data["production_countries"].fillna(value="Sin Datos", inplace=True)

In [ ]:
data["production_countries"] = data["production_countries"].astype(str)

In [ ]:
data["budget"].replace("", inplace=True)
data["revenue"].replace("", inplace=True)

In [ ]:
data['budget'] = data['budget'].astype(float)

In [ ]:
data['revenue'] = data['revenue'].astype(float)

In [ ]:
#convierto los valores de la columna las siguientes columnas en valores flotantes para poder realizar la duvision ya que estaban como objetos
def calcular_retor(row):
    revenue = row['revenue']
    budget = row['budget']
    
    if pd.notnull(revenue) and pd.notnull(budget) and budget != 0:
        return revenue / budget
    else:
        return 0

In [ ]:
data['return'] = data.apply(calcular_retor, axis=1)

In [ ]:
def peliculas_mes(mes: str): 
    '''Se ingresa el mes y la funcion retorna la cantidad de peliculas que se estrenaron 
        ese mes 
    ''' 
#nos imprimira el ideoma que le indiquemos  
    espanol_ingles = {
     'enero': 'January',
     'febrero': 'February',
     'marzo': 'March',
     'abril': 'April',
     'mayo': 'May',
     'junio': 'June',
     'julio': 'July',
     'agosto': 'August',
     'septiembre': 'September',
     'octubre': 'October',
     'noviembre': 'November',
     'diciembre': 'December'}
   
    if mes.lower() in espanol_ingles:
        mes_medi = espanol_ingles[mes.lower()]
    elif mes.lower not in espanol_ingles:
        mes_medi = mes
        
    #Convertimos a la columna de tipo datetime
    data["release_date"] = pd.to_datetime(data["release_date"])
    
    #Obtenemos mes y año de la columna
    data['mes'] = data['release_date'].dt.month_name() 
    data['año'] = data['release_date'].dt.year

    #Filtramos por el mes proporcionado
    peliculas_mes = data[data['mes'].str.lower() == mes_medi.lower()]
    
    #Cantidada de pelicula por mes 
    cantidad = len(peliculas_mes)
    
    return {'mes':mes, 'cantidad':cantidad}

In [ ]:
x = peliculas_mes("enero")
print(x)

In [76]:
def peliculas_dia(dia:str):
    '''
    Se ingresa el dia y la funcion retorna la cantidad de peliculas que se 
    estrenaron ese dia 

    ''' 
    # Convertir la columna a tipo datetime 
    data['release_date'] = pd.to_datetime(data['release_date']) 

    # Obtener el día de la semana de la columna
    data['dia_semana'] = data['release_date'].dt.day_name()

    # Filtrar  por el día de la semana proporcionado
    peliculas_dia = data[data['dia_semana'].str.lower() == dia.lower()]
 
    cantidad = len(peliculas_dia) 
    return  {'dia': dia, 'cantidad': cantidad}

In [77]:
x = peliculas_dia("Monday")
print (x)

{'dia': 'Monday', 'cantidad': 3503}


In [ ]:
def franquicia(franquicia: str):
    '''
    Se ingresa la franquicia, retornando la 
    cantidad de peliculas, ganancia total y promedio
    '''
    # Filtro el DataFrame por la franquicia proporcionada
    franquicia_fil = data[data['belongs_to_collection'].notnull() & data['belongs_to_collection'].apply(lambda x: isinstance(x, str) and franquicia.lower() in x.lower())]

    cantidad = len(franquicia_fil)

    ganancia_en_total = franquicia_fil['revenue'].sum()-franquicia_fil['budget'].sum()
    ganancia_en_promedio = ganancia_en_total/cantidad

    return {'franquicia': franquicia, 'cantidad': cantidad, 'ganancia_total': ganancia_en_total , 'ganancia_promedio': ganancia_en_promedio}

In [ ]:
respuesta = franquicia('Father of the Bride Collection')
print(respuesta)

In [ ]:
def peliculas_pais(pais: str):
    '''
    Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo
    '''
    # Filtrar la data por el país proporcionado
    pelicula_pais = data[data["production_countries"].str.contains(pais, case=False, na=False)]


    # cantidad de películas producidas en el país
    cantidad = len(pelicula_pais)

    return {'pais': pais, 'cantidad': cantidad}

In [ ]:
x = peliculas_pais("United States of America")
print(x)

In [ ]:
def productoras(productora: str):
    '''
    Ingresas la productora, retornando la ganancia total y 
    la cantidad de peliculas que produjeron
    '''
    # Filtro por la productora proporcionada
    productora_fil = data[data['production_companies'].notnull() & data['production_companies'].str.contains(productora, case=False)]

    # Obtener la ganancia total y la cantidad de películas producidas por la productora
    ganancia_en_total = productora_fil['revenue'].sum()-productora_fil['budget'].sum()
    cantidad = len(productora_fil)

    return {'productora': productora, 'ganancia_total': ganancia_en_total, 'cantidad': cantidad}

In [ ]:
x = productoras("Interscope Communications")
print(x)

In [ ]:
def retorno(pelicula: str): 
    '''Ingresas la pelicula, retornando la inversion, la ganancia, 
    el retorno y el año en el que se lanzo
    ''' 
    #filtramos las filas correspondientes 
    fila_pelicula = data[data["title"] == pelicula].iloc[0]
    
    #sacamos los valores 
    inversion = float(fila_pelicula["budget"])
    ganancia = float(fila_pelicula["revenue"]) - inversion
    retorno = float(fila_pelicula["return"])
    anio = int(fila_pelicula["release_year"])

    return {'pelicula':pelicula, 'inversion':inversion, 'ganacia':ganancia,'retorno': retorno, 'anio': anio}

In [ ]:
x = retorno("Jumanji")
print(x)